## London borough with the best House Price (Terraced) to Crime ratio

### Data Sources

##### London Crime data by borough from Sept 2016 - Aug 2018
https://data.london.gov.uk/dataset/recorded_crime_summary


##### London House Prices data by borough from Jan 2016 - July 2018
https://data.london.gov.uk/dataset/uk-house-price-index

In [185]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [186]:
# Load data
crime_raw_df = pd.read_csv('./data/MPS Borough Level Crime.csv')
house_price_raw_df = pd.read_excel('./data/UK House price index.xls', sheet_name='Average price')

### Format Crime Data

In [187]:
# Keep the required columns
keep_columns = [
    'Borough', 
    '201701', '201702', '201703', '201704', '201705', '201706', 
    '201707', '201708', '201709', '201710', '201711', '201712',
    '201801', '201802', '201803', '201804', '201805', '201806',
    '201807'
]

crime_raw_df = crime_raw_df[keep_columns]

In [188]:
# Group and sum all crimes by borough
crime_df = crime_raw_df.groupby('Borough').sum()

# Drop unwanted row
crime_df = crime_df.drop('London Heathrow and London City Airports')

In [189]:
# Stack data and name series
crime_df = crime_df.stack()
crime_df.name = 'Crime'

### Format House Price Data

In [190]:
# Keep the required rows and columns
columns = [
    'Barking & Dagenham', 'Barnet', 'Bexley', 'Brent',
    'Bromley', 'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich',
    'Hackney', 'Hammersmith & Fulham', 'Haringey', 'Harrow', 'Havering',
    'Hillingdon', 'Hounslow', 'Islington', 'Kensington & Chelsea',
    'Kingston upon Thames', 'Lambeth', 'Lewisham', 'Merton', 'Newham',
    'Redbridge', 'Richmond upon Thames', 'Southwark', 'Sutton',
    'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster'
]
house_price_raw_df = house_price_raw_df[columns]

# Get data from Jan 2017 to Aug 2018
house_price_raw_df = house_price_raw_df.loc['2017-01-01': '2018-07-01']

# Change index to year month
house_price_raw_df.index = house_price_raw_df.index.map(lambda x: x.strftime('%Y%m'))

In [191]:
# Stack data and name series
house_price_df = house_price_raw_df.T.stack()
house_price_df.name = 'House Price'

### Merge Crime and House Price Data

In [192]:
crime_df.head()

Borough                     
Barking and Dagenham  201701    1379
                      201702    1410
                      201703    1604
                      201704    1497
                      201705    1706
Name: Crime, dtype: int64

In [193]:
house_price_df.head()

Barking & Dagenham  201701    282441
                    201702    280698
                    201703    278213
                    201704    280567
                    201705    286018
Name: House Price, dtype: object

In [194]:
# Validate data shapes
crime_df.shape == house_price_df.shape

True

In [195]:
type(crime_df) == type(house_price_df)

True

In [196]:
# Convert series to dataframe
crime_df = pd.DataFrame(crime_df)
house_price_df = pd.DataFrame(house_price_df)

# Name multi index


In [197]:
crime_df.index.names = ['Borough', 'Period']
house_price_df.index.names = ['Borough', 'Period']

In [204]:
final_df = pd.merge(crime_df, house_price_df, left_index=True, right_index=True)
final_df.head()

Crime House Price
Borough Period                   
Barnet  201701   2077      532924
        201702   2045      540501
        201703   2438      549401
        201704   2207      543699
        201705   2293      544994

In [206]:
final_df.xs('Barnet').mean()

Crime            2272.789474
House Price    536771.343300
dtype: float64